In [ ]:
# %pip uninstall torch torchvision torchaudio -y
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [1]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")
print("Cahcle:", torch.cuda.empty_cache())

CUDA available: True
Device: NVIDIA GeForce RTX 3050 Laptop GPU
Cahcle: None


In [9]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
import os
import numpy as np

In [6]:
# === CONFIG ===
IMG_DIR_TRAIN = r"D:/School/Semester 8/PENGANTAR DEEP LEARNING/UTS/final-dataset-1/train"
IMG_DIR_VAL = r"D:/School/Semester 8/PENGANTAR DEEP LEARNING/UTS/final-dataset-1/valid"
IMG_DIR_TEST = r"D:/School/Semester 8/PENGANTAR DEEP LEARNING/UTS/final-dataset-1/test"
CSV_PATH_TRAIN = r"D:/School/Semester 8/PENGANTAR DEEP LEARNING/UTS/final-dataset-1/train/_classes.csv"
CSV_PATH_VAL = r"D:\School\Semester 8/PENGANTAR DEEP LEARNING/UTS/final-dataset-1/valid/_classes.csv"
CSV_PATH_TEST = r"D:/School/Semester 8/PENGANTAR DEEP LEARNING/UTS/final-dataset-1/test/_classes.csv"

BATCH_SIZE = 8
EPOCHS = 16
IMG_SIZE = 224
LR = 1e-4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)

Device: cuda


In [10]:
# === CLASSES ===
CLASSES = [
    "front_left_door_closed", "front_left_door_open",
    "front_right_door_closed", "front_right_door_open",
    "hood_closed", "hood_open",
    "rear_left_door_closed", "rear_left_door_open",
    "rear_right_door_closed", "rear_right_door_open"
]
CLASS2IDX = {c: i for i, c in enumerate(CLASSES)}

# === Dataset ===
class MultiLabelDataset(Dataset):
    def __init__(self, csv_path, img_dir, transform=None):
        self.data = pd.read_csv(csv_path)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        img_path = os.path.join(self.img_dir, row["filename"])
        image = Image.open(img_path).convert("RGB")

        label_vec = torch.tensor(row[CLASSES].values.astype(np.float32))

        if self.transform:
            image = self.transform(image)

        return image, label_vec


# === Transforms ===
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
])

# === DataLoader ===
def load_dataset(csv_path, img_dir):
    return MultiLabelDataset(csv_path, img_dir, transform)

# Load the datasets
train_dataset = load_dataset(CSV_PATH_TRAIN, IMG_DIR_TRAIN)
val_dataset = load_dataset(CSV_PATH_VAL, IMG_DIR_VAL)
test_dataset = load_dataset(CSV_PATH_TEST, IMG_DIR_TEST)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# === Model ===
model = models.resnet18(pretrained=True)
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, len(CLASSES)),
    nn.Sigmoid()  # For multi-label classification
)
model = model.to(DEVICE)

# === Optimizer & Loss ===
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

# === Training Loop ===
for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    for images, targets in train_loader:
        images, targets = images.to(DEVICE), targets.to(DEVICE)
        outputs = model(images)
        loss = criterion(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for images, targets in val_loader:
            images, targets = images.to(DEVICE), targets.to(DEVICE)
            outputs = model(images)
            loss = criterion(outputs, targets)
            val_loss += loss.item()

    print(f"[Epoch {epoch+1}/{EPOCHS}] 🔁 Train Loss: {train_loss/len(train_loader):.4f} | ✅ Val Loss: {val_loss/len(val_loader):.4f}")

# === Save model ===
torch.save(model.state_dict(), "multi_label_carparts_resnet18.pt")
print("✅ Model saved!")


d:\School\Semester 8\PENGANTAR DEEP LEARNING\UTS\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\School\Semester 8\PENGANTAR DEEP LEARNING\UTS\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[Epoch 1/16] 🔁 Train Loss: 0.4557 | ✅ Val Loss: 0.2997
[Epoch 2/16] 🔁 Train Loss: 0.2852 | ✅ Val Loss: 0.2192
[Epoch 3/16] 🔁 Train Loss: 0.2059 | ✅ Val Loss: 0.1971
[Epoch 4/16] 🔁 Train Loss: 0.1561 | ✅ Val Loss: 0.2039
[Epoch 5/16] 🔁 Train Loss: 0.1193 | ✅ Val Loss: 0.1787
[Epoch 6/16] 🔁 Train Loss: 0.0727 | ✅ Val Loss: 0.2150
[Epoch 7/16] 🔁 Train Loss: 0.0573 | ✅ Val Loss: 0.1621
[Epoch 8/16] 🔁 Train Loss: 0.0477 | ✅ Val Loss: 0.1706
[Epoch 9/16] 🔁 Train Loss: 0.0383 | ✅ Val Loss: 0.1553
[Epoch 10/16] 🔁 Train Loss: 0.0298 | ✅ Val Loss: 0.3187
[Epoch 11/16] 🔁 Train Loss: 0.0184 | ✅ Val Loss: 0.1745
[Epoch 12/16] 🔁 Train Loss: 0.0189 | ✅ Val Loss: 0.2038
[Epoch 13/16] 🔁 Train Loss: 0.0348 | ✅ Val Loss: 0.1771
[Epoch 14/16] 🔁 Train Loss: 0.0271 | ✅ Val Loss: 0.1635
[Epoch 15/16] 🔁 Train Loss: 0.0174 | ✅ Val Loss: 0.2246
[Epoch 16/16] 🔁 Train Loss: 0.0132 | ✅ Val Loss: 0.1301
✅ Model saved!


In [ ]:
from ultralytics import YOLO

model = YOLO('yolo11n.pt')  # smaller model = less overfitting

results = model.train(
    data="/content/yolo-segmantation-2-1/data.yaml",
    epochs=16,
    batch=8,
    device=0,
    task='detect',
    name='car-prediction-state-1',
    exist_ok=False
)

In [ ]:
import os
import subprocess

# === PATH CONFIGURATION ===
YOLOV11_DIR = "path/to/yolo-v11"  # change this to your yolo-v11 clone path
DATASET_DIR = r"D:/School/Semester 8/PENGANTAR DEEP LEARNING/UTS/dataset-D"

# === CREATE dataset.yaml ===
dataset_yaml_path = os.path.join(YOLOV11_DIR, "dataset.yaml")

dataset_yaml_content = f"""
path: {DATASET_DIR.replace('\\', '/')}
train: train
val: val

nc: 10
names:
  - front_left_door_closed
  - front_left_door_open
  - front_right_door_closed
  - front_right_door_open
  - hood_closed
  - hood_open
  - rear_left_door_closed
  - rear_left_door_open
  - rear_right_door_closed
  - rear_right_door_open
"""

with open(dataset_yaml_path, "w") as f:
    f.write(dataset_yaml_content)

print("✅ dataset.yaml generated.")

# === RUN TRAINING ===
train_command = [
    "python",
    "train.py",
    "--model", "yolo11n.pt",         # or use yolo11s.pt / yolo11m.pt
    "--task", "detect",
    "--data", "dataset.yaml",
    "--imgsz", "640",
    "--epochs", "100",
    "--batch", "16",
    "--device", "0"
]

print("🚀 Starting YOLOv11 training...\n")

subprocess.run(train_command, cwd=YOLOV11_DIR)

print("\n✅ Training finished.")
